In [1]:
import auth
import requests
import base64
import json
import pandas as pd
flickr = auth.connect()

In [2]:
df = pd.read_csv('../us_cities.csv')


top_cities = ["New York", "Los Angeles", "Las Vegas", "Orlando", "Miami", "San Francisco"]

df_top = df[df['CITY'].isin(top_cities)]
df_top = df_top.reset_index()  

In [3]:
cities= {
   "New York":{
      "latitude":40.730610,
      "longitude":-73.935242
   },
   "Los Angeles":{
      "latitude":34.052235,
      "longitude":-118.243683
   },
   "Las Vegas":{
      "latitude":36.188110,
      "longitude":-115.176468
   },
   "Orlando":{
      "latitude":28.538336,
      "longitude":-81.379234
   },
   "Miami":{
      "latitude":25.761681,
      "longitude":-80.191788
   },
   "San Francisco":{
      "latitude":37.773972,
      "longitude":-122.431297
   }
}


In [4]:
# cities= {
#    "New York":{
#       "latitude":40.730610,
#       "longitude":-73.935242
#    }
# }

Fields needed for each photo:
id: unique ID of the uploaded photo (pid).
owner-id: unique user ID of the person who uploaded the photo (up).
longitude: (geotag information) x coordinate (xp).
latitude: (geotag information) y coordinate (yp).
dates-taken: date and time when the photo was taken (tp).
dates-posted: date and time when the photo was uploaded.

In [5]:
def getlonglat(row):
    res = flickr.photos.geo.getLocation(photo_id=str(row['id']))
    
    location = res['photo']['location']
    row['latitude'] = location['latitude']
    row['longitude'] = location['longitude']
    return row


In [6]:
for k,v in cities.items():
    data=[]
    pages = 8
    city = k
    long = v['longitude']
    lat = v['latitude']
    filename= city+'-photos.csv'

    for p in range(1, pages+1):
        
    #     photos = flickr.photos.search(lat=lat, lon=long, accuracy=11,min_date_taken='2010-01-01', max_date_taken='2023-12-31', radius = 20, radius_units='mi', extras=["date_upload", "date_taken", "owner_name", "geo"])
        photos = flickr.photos.search(lat=lat, lon=long, accuracy=11, radius = 20, radius_units='mi',min_date_taken='2010-01-01', max_date_taken='2023-12-31', extras=["date", "date_upload", "date_taken", "owner_name"], page=p)

        df_photos = pd.DataFrame(photos['photos']['photo'])

        df_photos = df_photos.apply(getlonglat, axis=1)
        data.append(df_photos)
    data = pd.concat(data)
    data.to_csv(filename)


 

In [7]:
data

,id,owner,secret,server,farm,title,ispublic,isfriend,isfamily,ownername,latitude,longitude
0,52717572991,124549018@N05,990eda0d3b,65535,66,New-York,1,0,0,Martin Cousture,40.768188,-73.973359
1,52716985534,7735863@N05,a0811b73ce,65535,66,"the orchid show: natural heritage, 2/26/23",1,0,0,hollow sidewalks,40.864863,-73.906162
2,52717148790,7735863@N05,76d7e2bf96,65535,66,"the orchid show: natural heritage, 2/26/23",1,0,0,hollow sidewalks,40.864863,-73.882337
3,52715472564,48307030@N02,0fd8d74938,65535,66,0009138bw,1,0,0,Naturecamhd,40.888823,-73.876009
4,52715249533,69233611@N05,fac9bae049,65535,66,New York City 2023,1,0,0,broadswordcallingdannyboy,40.780037,-73.982137
...,...,...,...,...,...,...,...,...,...,...,...,...
245,51744182192,11086755@N00,e88a1a5867,65535,66,Central Park South,1,0,0,Chicamguy,40.766497,-73.976556
246,51741692137,70323761@N00,aaca776564,65535,66,Jersey City: Bones by ELLE,1,0,0,wallyg,40.730713,-74.046289
247,51740840267,9549670@N05,63d5e9d6a5,65535,66,"New York City, NY - Manhattan - Metropolitan M...",1,0,0,jrozwado,40.779521,-73.963029
248,51741474176,37580254@N03,64defb4c13,65535,66,Walk in Brooklyn,1,0,0,DAN GAKEN IMAGES,40.702008,-73.995355


In [8]:
df1= data.drop_duplicates()

In [9]:
df1

,id,owner,secret,server,farm,title,ispublic,isfriend,isfamily,ownername,latitude,longitude
0,52717572991,124549018@N05,990eda0d3b,65535,66,New-York,1,0,0,Martin Cousture,40.768188,-73.973359
1,52716985534,7735863@N05,a0811b73ce,65535,66,"the orchid show: natural heritage, 2/26/23",1,0,0,hollow sidewalks,40.864863,-73.906162
2,52717148790,7735863@N05,76d7e2bf96,65535,66,"the orchid show: natural heritage, 2/26/23",1,0,0,hollow sidewalks,40.864863,-73.882337
3,52715472564,48307030@N02,0fd8d74938,65535,66,0009138bw,1,0,0,Naturecamhd,40.888823,-73.876009
4,52715249533,69233611@N05,fac9bae049,65535,66,New York City 2023,1,0,0,broadswordcallingdannyboy,40.780037,-73.982137
...,...,...,...,...,...,...,...,...,...,...,...,...
245,51744182192,11086755@N00,e88a1a5867,65535,66,Central Park South,1,0,0,Chicamguy,40.766497,-73.976556
246,51741692137,70323761@N00,aaca776564,65535,66,Jersey City: Bones by ELLE,1,0,0,wallyg,40.730713,-74.046289
247,51740840267,9549670@N05,63d5e9d6a5,65535,66,"New York City, NY - Manhattan - Metropolitan M...",1,0,0,jrozwado,40.779521,-73.963029
248,51741474176,37580254@N03,64defb4c13,65535,66,Walk in Brooklyn,1,0,0,DAN GAKEN IMAGES,40.702008,-73.995355
